# Simple Hugging Face TTS

In [ ]:

# import torch
# from transformers import CsmForConditionalGeneration, AutoProcessor
# from huggingface_hub import snapshot_download
# import torchaudio
# import os

# # Option 1: Pre-download model for offline use
# # Uncomment to download model ahead of time for offline/firewalled environments
# # snapshot_download(repo_id="sesame/csm-1b", repo_type="model")

# # Option 2: Set environment variable for offline usage
# # os.environ["HF_HUB_OFFLINE"] = "1"

# # 1. Model & processor
# model_id = "sesame/csm-1b"
# device = "cuda" if torch.cuda.is_available() else "cpu"

# # Option 3: Use local_files_only=True for cached/offline models
# processor = AutoProcessor.from_pretrained(model_id)  # Add local_files_only=True if using cached model
# model = CsmForConditionalGeneration.from_pretrained(model_id, device_map=device)  # Add local_files_only=True if using cached model

# # 2. Prepare input text (speaker id 0)
# text = "[0]Hello from Sesame!"
# inputs = processor(text, return_tensors="pt").to(device)

# # 3. Synthesize audio
# audio = model.generate(**inputs, output_audio=True)

# # 4. Save as WAV
# torchaudio.save("output.wav", audio.unsqueeze(0).cpu(), processor.feature_extractor.sampling_rate)






ModuleNotFoundError: No module named 'models'

# Eleven Labs Streaming Audio Only

In [2]:
from elevenlabs import stream
from elevenlabs.client import ElevenLabs
import os
from dotenv import load_dotenv
load_dotenv()
key = os.getenv('ELEVEN_LABS_KEY')
client = ElevenLabs(api_key=key)
audio_stream = client.text_to_speech.stream(
    #text="This is a test",
    text = '''★ شادی شدہ کی موت پر دوسرے کی شادی کرنا ۔ (عدت کے بعد)
★ شوہر اپنی عورت کا جنازہ پڑھائے ۔
★ عورت کی میت کو غیر مرد ہاتھ لگائے ۔ (عدت کے اندر بھی)
★ زنا کاری ۔ بے غیرتی ۔ خود غرضی ۔ چوری ۔ سود ۔ حرام ۔
★ زنا کاری کی سزا ۔ (عورت کا سر مونڈنا، پتھر مار مار کر ہلاک کرنا ۔) (مسلم)

  ''',
    voice_id="JBFqnCBsd6RMkjVDRZzb",
    model_id="eleven_multilingual_v2"
)

# option 1: play the streamed audio locally
stream(audio_stream)

# option 2: process the audio bytes manually
for chunk in audio_stream:
    if isinstance(chunk, bytes):
        print(chunk)


# Eleven Labs TTS on OCR Extracted Text

In [1]:
from elevenlabs import stream
from elevenlabs.client import ElevenLabs
import os
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables
load_dotenv()
key = os.getenv('ELEVEN_LABS_KEY')
client = ElevenLabs(api_key=key)

def convert_text_to_audio(text_file_path, output_dir="tts_outputs"):
    """
    Convert a text file to audio and save it
    """
    try:
        # Read the text file
        with open(text_file_path, 'r', encoding='utf-8') as file:
            text_content = file.read()
        
        if not text_content.strip():
            print(f"Skipping empty file: {text_file_path}")
            return
        
        # Print the text content being processed
        print(f"\n{'='*60}")
        print(f"PROCESSING FILE: {Path(text_file_path).name}")
        print(f"{'='*60}")
        print(f"TEXT CONTENT:\n{text_content}")
        print(f"{'='*60}\n")
        
        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        # Generate audio filename
        text_filename = Path(text_file_path).stem
        audio_filename = f"{text_filename}.mp3"
        audio_path = os.path.join(output_dir, audio_filename)
        
        print(f"Converting: {text_file_path} -> {audio_path}")
        
        # Generate audio stream
        audio_stream = client.text_to_speech.stream(
            text=text_content,
            voice_id="JBFqnCBsd6RMkjVDRZzb",
            model_id="eleven_multilingual_v2"
        )
        
        # Save audio to file and collect chunks for streaming
        audio_chunks = []
        with open(audio_path, 'wb') as audio_file:
            for chunk in audio_stream:
                if isinstance(chunk, bytes):
                    audio_file.write(chunk)
                    audio_chunks.append(chunk)
        
        print(f"Audio saved: {audio_path}")
        
        # Play the audio aloud
        print(f"Playing audio: {audio_filename}")
        try:
            # Create a generator from the collected chunks to stream
            def audio_generator():
                for chunk in audio_chunks:
                    yield chunk
            
            stream(audio_generator())
            print(f"Finished playing: {audio_filename}")
        except Exception as e:
            print(f"Error playing audio {audio_filename}: {str(e)}")
            # Fallback: try to play from saved file
            try:
                with open(audio_path, 'rb') as audio_file:
                    audio_data = audio_file.read()
                    stream(audio_data)
                print(f"Finished playing (fallback): {audio_filename}")
            except Exception as e2:
                print(f"Fallback audio playback also failed for {audio_filename}: {str(e2)}")
        
    except Exception as e:
        print(f"Error processing {text_file_path}: {str(e)}")

def process_ollama_direct_folder(folder_path="extracted_text/ollama_direct", output_dir="tts_outputs"):
    """
    Process all text files in the ollama_direct folder
    """
    folder_path = Path(folder_path)
    
    if not folder_path.exists():
        print(f"Folder {folder_path} does not exist!")
        return
    
    # Find all .txt files in the folder
    txt_files = list(folder_path.glob("*.txt"))
    
    if not txt_files:
        print(f"No .txt files found in {folder_path}")
        return
    
    print(f"Found {len(txt_files)} text files to process")
    
    # Process each text file
    for txt_file in txt_files:
        convert_text_to_audio(txt_file, output_dir)
        print("-" * 50)

# Main execution
if __name__ == "__main__":
    # Process all text files in ollama_direct folder
    process_ollama_direct_folder()
    
    print("All files processed successfully!")

Found 3 text files to process

PROCESSING FILE: textbook_img.txt
TEXT CONTENT:
The text in the image reads:

"Episode 25: When the Moon Splits

Synopsis:

The war for Ryloth rages on, as Mon Calamari rebels are forced to confront the reality of their situation.

In the midst of the battle, Chancellor Lando Calrissian must decide whether to continue with his risky plan or abandon it. The fate of the moon hangs in the balance.

The crew of the Ghost, led by a reluctant Hera Syndulla, uncovers an unexpected ally in their quest for victory. But can they trust this newcomer?

Meanwhile, Sabine's skills as a weapons smith are put to the test as she attempts to repair the damage done during the battle.

As the moon splits, Hera must lead her team through a gauntlet of challenges in order to save their home and loved ones.

Will they be able to overcome their fears and save Ryloth? Or will it slip into darkness forever?

The fate of the moon and its inhabitants rests in the hands of the Ghost 

KeyboardInterrupt: 